In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import sklearn as sk
import torch.nn as nn                     
import torch.nn.functional as F
import torch.optim as optim
import pickle
import xlwt
from collections import Counter 
from xlwt import Workbook 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from random import randint
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, LSTM, Lambda, BatchNormalization, GaussianNoise, Flatten
from keras.layers.merge import Add, Multiply
from keras.optimizers import Adam
import keras.backend as KB
import random
from collections import deque
from keras.initializers import RandomUniform
from tqdm import tqdm
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from keras.layers import concatenate
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier

In [ ]:
path = '/content/gdrive/MyDrive/New Dataset/'
benign_src = path + 'Permission_benign.csv'
malware_src = path + 'Permission_malware.csv'

In [ ]:
benign = pd.read_csv(benign_src)
malware = pd.read_csv(malware_src)
# Add Class Label to Benign and Malware Instances
benign['Class'] = 0
malware['Class'] = 1
print(benign.info())
print(malware.info())

In [ ]:
# Rename 1st Column as ID
malware.rename( columns={'Unnamed: 0':'ID'}, inplace=True )
benign.rename( columns={'Unnamed: 0':'ID'}, inplace=True )

In [ ]:
# Combine Benign and Malware Instances
combined = pd.concat([benign, malware],ignore_index=True)
print(combined.info)
permission_number = 0
permissions = {}
for column in combined.columns:
  if column != 'ID' and column != 'Class':
    permissions[permission_number] = column
    permission_number = permission_number + 1
print(permissions)

In [ ]:
# Split Data into X and Y
X = combined.loc[:,combined.columns != 'Class']
X = X.loc[:,X.columns != 'ID']
Y = combined.loc[:, combined.columns == 'Class']
X = pd.DataFrame(X).to_numpy()
Y = pd.DataFrame(Y).to_numpy()
print(X.shape)
print(Y.shape)

**Define Classifiers**

In [ ]:
def classifyRF(X_train,X_test,Y_train,Y_test):
  # Define Random Forest Classifier and Fit on Data
  rf_classifier = RandomForestClassifier()
  rf_classifier = rf_classifier.fit(X_train,Y_train)
  Y_pred = rf_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (rf_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifyDT(X_train,X_test,Y_train,Y_test):
  # Define Decision Tree Classifier and Fit on Data
  dt_classifier = tree.DecisionTreeClassifier()
  dt_classifier = dt_classifier.fit(X_train,Y_train)
  Y_pred = dt_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (dt_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifyBNB(X_train,X_test,Y_train,Y_test):
  # Define Bernoulli Naive Bayes Classifier and Fit on Data
  bnb_classifier = BernoulliNB()
  bnb_classifier = bnb_classifier.fit(X_train,Y_train)
  Y_pred = bnb_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (bnb_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifySVM(X_train,X_test,Y_train,Y_test):
  # Define Support Vector Machine Classifier and Fit on Data
  svm_classifier = svm.SVC(probability=True)
  svm_classifier = svm_classifier.fit(X_train,Y_train)
  Y_pred = svm_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (svm_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifyLR(X_train,X_test,Y_train,Y_test):
  # Define Logistic Regression Classifier and Fit on Data
  lr_classifier = LogisticRegression()
  lr_classifier = lr_classifier.fit(X_train,Y_train)
  Y_pred = lr_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (lr_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifyGB(X_train,X_test,Y_train,Y_test):
  # Define Gradient Boost Classifier and Fit on Data
  gb_classifier = GradientBoostingClassifier()
  gb_classifier = gb_classifier.fit(X_train,Y_train)
  Y_pred = gb_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (gb_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifyXGB(X_train,X_test,Y_train,Y_test):
  # Define Extreme Gradient Boost Classifier and Fit on Data
  xgb_classifier = xgb.XGBClassifier()
  xgb_classifier = xgb_classifier.fit(X_train,Y_train)
  Y_pred = xgb_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (xgb_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifyStack(X_train,X_test,Y_train,Y_test):
  # Define Stacked Classifier and Fit on Data
  estimators = [('svm',svm.SVC()),
                ('dt',tree.DecisionTreeClassifier())]
  stack_classifier = StackingClassifier(estimators=estimators)
  stack_classifier = stack_classifier.fit(X_train,Y_train)
  Y_pred = stack_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (stack_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifyDNN1(X_train,X_test,Y_train,Y_test):
  # Define Deep Neural Network Classifier with 1 hidden layer and Fit on Data
  dnn_classifier = MLPClassifier(hidden_layer_sizes=(64),max_iter=350)
  dnn_classifier = dnn_classifier.fit(X_train,Y_train)
  Y_pred = dnn_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (dnn_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifyDNN3(X_train,X_test,Y_train,Y_test):
  # Define Deep Neural Network Classifier with 3 hidden layer and Fit on Data
  dnn_classifier = MLPClassifier(hidden_layer_sizes=(64,32,16))
  dnn_classifier = dnn_classifier.fit(X_train,Y_train)
  Y_pred = dnn_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (dnn_classifier,accuracy,precision,recall,f1, Y_pred)

In [ ]:
def classifyDNN5(X_train,X_test,Y_train,Y_test):
  # Define Deep Neural Network Classifier with 5 hidden layer and Fit on Data
  dnn_classifier = MLPClassifier(hidden_layer_sizes=(128,64,32,16,8))
  dnn_classifier = dnn_classifier.fit(X_train,Y_train)
  Y_pred = dnn_classifier.predict(X_test)
  # Return Stats
  accuracy = accuracy_score(Y_test,Y_pred)
  precision = precision_score(Y_test,Y_pred)
  recall = recall_score(Y_test,Y_pred)
  f1 = f1_score(Y_test,Y_pred)
  return (dnn_classifier,accuracy,precision,recall,f1, Y_pred)

**Define Environment for RL**

In [ ]:
class Env():
  def __init__(self,num_features,model):
    # Define Classifier and Number of Features Used
    self.num_features = num_features
    self.model = model
    # Extract only Malware Instances
    #(self.X,self.Y) = self.correlation_select(self.num_features)
    #(self.X,self.Y) = self.univariate_select(self.num_features)
    self.X = np.copy(X)
    self.Y = np.copy(Y)
    self.indices = np.where(self.Y == 1)
    self.data = self.X[self.indices[0]]
    # Split Malware Instances into Train and Test Data
    #self.train_size = int(np.rint(self.data.shape[0] * 0.8))
    #self.train_data = self.data[:self.train_size,:]
    #self.test_data = self.data[self.train_size:,:]
    self.train_data = np.copy(self.data)
    self.test_data = np.copy(self.data)
    # Define Observation Space, Action Space, State, Action, Reward Variables
    self.observation_space = num_features
    self.action_space = num_features
    self.action_range = 1
    self.state = np.zeros((self.num_features,1))
    self.action = randint(0,self.action_space)
    self.selected = np.zeros((self.train_data.shape[0],1))
    self.done = 0
    self.reward = 0
    self.num = 0
  
  def univariate_select(self,num_features):
    # Choose Top K Features Based on Chi Squared Test
    selector = SelectKBest(chi2, k=num_features)
    combined_topK = combined.loc[:,combined.columns != 'Class']
    combined_topK = combined_topK.loc[:,combined_topK.columns != 'ID']
    Y_topK = combined.loc[:,combined.columns == 'Class']
    X_topK = selector.fit_transform(combined_topK,Y_topK)
    return (X_topK,Y_topK)

  def correlation_select(self,num_features):
    # Choose Top K Features Based on Correlation with Class Label
    X = combined.loc[:,combined.columns != 'Class']
    X = X.loc[:,X.columns != 'ID']
    X = X.loc[:, (X != 0).any(axis=0)]
    Y = combined.loc[:, combined.columns == 'Class']
    X = pd.DataFrame(X).to_numpy()
    Y = pd.DataFrame(Y).to_numpy()
    XY = np.concatenate((X,Y),axis = 1)
    corr = np.corrcoef(XY.T)
    target_corr = corr[:][-1]
    target_corr = np.absolute(target_corr)
    target_corr = target_corr[0:len(target_corr) - 1]
    index = target_corr.argsort()
    X_topK = X[:,index[::-1][:num_features]]
    Y_topK = Y
    return (X_topK,Y_topK)
  
  def reset(self):
    # Start New Episode of Training by Choosing a New Malware Instance Randomly
    self.done = 0
    while True:
      # If all Instances Seen, Reset All to Unseen and Choose Randomly Again
      result = np.max(self.selected) == np.min(self.selected)
      if result:
        self.selected = np.zeros((self.train_data.shape[0],1))
      self.num = randint(0,self.train_data.shape[0] - 1)
      if(self.selected[self.num] == 0):
        self.selected[self.num] = 1
        self.state = self.train_data[self.num]
        break
      else:
        continue
    return self.state
  
  def step(self,action):
    # Take an Action
    prev_obs = np.zeros((self.num_features,))
    next_obs = np.zeros((self.num_features,))
    prev_obs[:] = self.state[:]
    # Penalise Agent for Taking Same Action Again and Return Next State, Action, Reward
    if self.state[action] == 1:
      (self.reward,self.done) = self.classify(prev_obs,prev_obs)
      self.reward = self.reward - 10
      return (self.state,self.reward,self.done)
    else:  
      self.state[action] = 1
      next_obs[:] = self.state[:]
      (self.reward,self.done) = self.classify(prev_obs,next_obs)
      return (self.state,self.reward,self.done)

  def classify(self,prev_obs,next_obs):
    # Return Change in Class Probabilities After Taking Action as Reward and Done Variable
    prev_obs = prev_obs.reshape(1, -1)
    next_obs = next_obs.reshape(1, -1)
    self.prev = self.model.predict_proba(prev_obs)[0]
    self.next = self.model.predict_proba(next_obs)[0]
    self.reward = self.next[0] - self.prev[0]
    if self.next[0] > 0.50:
      self.done = 1
    else:
      self.done = 0
    return (self.reward,self.done)


**Define Actor Network + Target Actor Network**

In [ ]:
class Actor():
    # Actor Network
    def __init__(self, inp_dim, out_dim, act_range, lr, tau):
        # Define Input Output Dimensions
        self.env_dim = inp_dim
        self.act_dim = out_dim
        self.act_range = act_range
        # Define Hyperparameters
        self.tau = tau
        self.lr = lr
        # Define Actor Network and Target Actor Network
        self.model = self.network()
        self.target_model = self.network()
        # Define Optimizer Used
        self.adam_optimizer = self.optimizer()

    def network(self):
        # Define Actor Network Architecture
        inp = Input(shape=(self.env_dim,))
        x = Dense(256, activation='relu')(inp)
        #x = GaussianNoise(1.0)(x)
        #x = Dense(80, activation='relu')(x)
        #x = GaussianNoise(1.0)(x)
        #x = Dense(160, activation='relu')(x)
        #x = Dense(80, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        #x = GaussianNoise(1.0)(x)
        out = Dense(self.act_dim, activation='softmax')(x)
        return Model(inp, out)

    def predict(self, state):
        # Predict Action Taken By Actor Network
        return self.model.predict(np.expand_dims(state, axis=0))

    def target_predict(self, inp):
        # Predict Action Taken By Target Actor Network
        return self.target_model.predict(inp)

    def transfer_weights(self):
        # Transfer Actor Network Weights to Target Actor Network with a Factor of Tau
        (W, target_W) = self.model.get_weights(), self.target_model.get_weights()
        for i in range(len(W)):
            target_W[i] = self.tau * W[i] + (1 - self.tau)* target_W[i]
        self.target_model.set_weights(target_W)

    def train(self, states, actions, grads):
        # Train Actor Network
        self.adam_optimizer([states, grads])

    def optimizer(self):
        # Define Optimizer
        action_gdts = KB.placeholder(shape=(None, self.act_dim))
        params_grad = tf.gradients(self.model.output, self.model.trainable_weights, -action_gdts)
        grads = zip(params_grad, self.model.trainable_weights)
        return KB.function(inputs=[self.model.input, action_gdts], outputs=[KB.constant(1)], updates= [tf.optimizers.Adam(self.lr).apply_gradients(grads)])

    def save(self, path):
        # Save Model Weights
        self.model.save_weights(path + '_actor.h5')

    def load_weights(self, path):
        # Load Model Weights
        self.model.load_weights(path)

**Define Critic Network + Target Critic Network**

In [ ]:
class Critic():
    # Critic Network
    def __init__(self, inp_dim, out_dim, lr, tau):
        # Define Input Output Dimensions
        self.env_dim = inp_dim
        self.act_dim = out_dim
        # Define Hyperparameters
        self.tau, self.lr = tau, lr
        # Define Critic Network and Target Critic Network
        self.model = self.network()
        self.target_model = self.network()
        self.model.compile(Adam(self.lr), 'mse')
        self.target_model.compile(Adam(self.lr), 'mse')
        # Function to Compute Q-value Gradients (Needed for Actor Optimization)
        self.action_grads = tf.keras.backend.function([self.model.input[0], self.model.input[1]], tf.keras.backend.gradients(self.model.output, [self.model.input[1]]))

    def network(self):
        # Define Critic Network Architecture
        state = Input(shape=(self.env_dim,))
        action = Input(shape=(self.act_dim,))
        x = Dense(256, activation='relu')(state)
        x = concatenate([x, action])
        x = Dense(128, activation='relu')(x)
        #x = Dense(160, activation='relu')(x)
        #x = Dense(80, activation='relu')(x)
        #x = Dense(40, activation='relu')(x)
        out = Dense(1, activation='linear', kernel_initializer=RandomUniform())(x)
        return Model([state, action], out)

    def gradients(self, states, actions):
        # Compute Q-value Gradients w.r.t. States and Actions
        return self.action_grads([states, actions])

    def target_predict(self, inp):
        # Predict Q Values Using Target Critic Network
        return self.target_model.predict(inp)

    def train_on_batch(self, states, actions, critic_target):
        # Train Critic Network
        return self.model.train_on_batch([states, actions], critic_target)

    def transfer_weights(self):
        # Transfer Critic Network Weights to Target Critic Network with a Factor of Tau
        (W, target_W) = self.model.get_weights(), self.target_model.get_weights()
        for i in range(len(W)):
            target_W[i] = self.tau * W[i] + (1 - self.tau)* target_W[i]
        self.target_model.set_weights(target_W)

    def save(self, path):
        # Save Model Weights
        self.model.save_weights(path + '_critic.h5')

    def load_weights(self, path):
        # Load Model Weights
        self.model.load_weights(path)

**Sum Tree Data Structure for Efficient Memory Usage**

In [ ]:
class SumTree():
    write = 0

    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros( 2*capacity - 1 )
        self.data = np.zeros( capacity, dtype=object )

    def _propagate(self, idx, change):
        parent = (idx - 1) // 2

        self.tree[parent] += change

        if parent != 0:
            self._propagate(parent, change)

    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1

        if left >= len(self.tree):
            return idx

        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s-self.tree[left])

    def total(self):
        return self.tree[0]

    def add(self, p, data):
        idx = self.write + self.capacity - 1

        self.data[self.write] = data
        self.update(idx, p)

        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

    def update(self, idx, p):
        change = p - self.tree[idx]

        self.tree[idx] = p
        self._propagate(idx, change)

    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1

        return (idx, self.tree[idx], self.data[dataIdx])
  

**Memory Buffer for Experience Replay**

In [ ]:
class MemoryBuffer(object):
    # Memory Buffer Class for Experience Replay Using Deque (Normal) or Sum Tree (Priority)
    def __init__(self, buffer_size, with_per = False):
        # Initialization
        if(with_per):
            # Prioritized Experience Replay
            self.alpha = 0.5
            self.epsilon = 0.01
            self.buffer = SumTree(buffer_size)
        else:
            # Standard Buffer
            self.buffer = deque()
        self.count = 0
        self.with_per = with_per
        self.buffer_size = buffer_size

    def memorize(self, state, action, reward, done, new_state, error=None):
        # Save an Experience to Memory, Optionally with its TD-Error 
        experience = (state, action, reward, done, new_state)
        if(self.with_per):
            priority = self.priority(error[0])
            self.buffer.add(priority, experience)
            self.count += 1
        else:
            # Check if Buffer is Already Full
            if self.count < self.buffer_size:
                self.buffer.append(experience)
                self.count += 1
            else:
                self.buffer.popleft()
                self.buffer.append(experience)

    def priority(self, error):
        # Compute Priority of Experience
        return (error + self.epsilon) ** self.alpha

    def size(self):
        # Return Current Size of Buffer
        return self.count

    def sample_batch(self, batch_size):
        # Sample a Batch
        batch = []

        # Sample using Prorities
        if(self.with_per):
            T = self.buffer.total() // batch_size
            for i in range(batch_size):
                a, b = T * i, T * (i + 1)
                s = random.uniform(a, b)
                idx, error, data = self.buffer.get(s)
                batch.append((*data, idx))
            idx = np.array([i[5] for i in batch])
        # Sample Randomly from Buffer
        elif self.count < batch_size:
            idx = None
            batch = random.sample(self.buffer, self.count)
        else:
            idx = None
            batch = random.sample(self.buffer, batch_size)

        # Return a Batch of Experiences
        s_batch = np.array([i[0] for i in batch])
        a_batch = np.array([i[1] for i in batch])
        r_batch = np.array([i[2] for i in batch])
        d_batch = np.array([i[3] for i in batch])
        new_s_batch = np.array([i[4] for i in batch])
        return s_batch, a_batch, r_batch, d_batch, new_s_batch, idx

    def update(self, idx, new_error):
        # Update Priority of Experiences
        self.buffer.update(idx, self.priority(new_error))

    def clear(self):
        # Clear Deque or Sum Tree
        if(self.with_per): self.buffer = SumTree(buffer_size)
        else: self.buffer = deque()
        self.count = 0

**Define Agent**

In [ ]:
class Agent():
    
    def __init__(self, env, buffer_size = 20000,batch_size = 64, epsilon = 1.00, epsilon_decay = 0.9995, gamma = 0.99, lr = 0.00005, tau = 0.001, with_per = False):
        # Define Environment and Agent Parameters
        self.env = env
        self.action_dim = self.env.action_space
        self.action_range = self.env.action_range
        self.observation_dim = self.env.observation_space
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma
        self.lr = lr
        self.tau = tau
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.with_per = with_per
        # Create Actor and Critic Networks
        self.actor = Actor(self.observation_dim,self.action_dim,self.action_range,0.1 * lr, tau)
        self.critic = Critic(self.observation_dim,self.action_dim, lr, tau)
        self.buffer = MemoryBuffer(self.buffer_size,self.with_per)

    def policy_action(self, s):
        # Predict Action Using Actor Network
        return self.actor.predict(s)[0]

    def bellman(self, rewards, q_values, dones):
        # Compute Critic Target Using Bellman Equation
        critic_target = np.asarray(q_values)
        for i in range(q_values.shape[0]):
            if dones[i]:
                critic_target[i] = rewards[i]
            else:
                critic_target[i] = rewards[i] + self.gamma * q_values[i]
        return critic_target

    def memorize(self, state, action, reward, done, new_state):
        # Store Experience in Buffer 
        self.buffer.memorize(state, action, reward, done, new_state)

    def sample_batch(self, batch_size):
        # Return Batch of Experiences
        return self.buffer.sample_batch(batch_size)

    def update_models(self, states, actions, critic_target):
        # Update Actor and Critic Models
        # Train Critic
        self.critic.train_on_batch(states, actions, critic_target)
        # Q-Value Gradients under Current Policy
        actions = self.actor.model.predict(states)
        grads = self.critic.gradients(states, actions)
        # Train Actor
        self.actor.train(states, actions, np.array(grads).reshape((-1, self.action_dim)))
        # Transfer Weights to Target Networks at Rate Tau
        self.actor.transfer_weights()
        self.critic.transfer_weights()

    def train(self,num_episodes):
      # Train Agent for a Specified Number of Episodes 
      num_alterations = []
      cumulative_rewards = []
      curr_state = np.zeros((self.observation_dim,))
      next_state = np.zeros((self.observation_dim,))
      for e in range(0,num_episodes):
        #print("Training Episode: ",e,self.epsilon)
        steps,cumulative_reward,done = 0,0,0
        curr_state[:] = self.env.reset()[:]
        while not done:
          randgen = random.random()
          #print(randgen)
          if(randgen < self.epsilon):
            act = np.random.rand(self.action_dim)
            s = np.sum(act)
            act = np.divide(act,s)
            #print(act)
            #print(np.sum(act))
            action = np.zeros((self.action_dim))
            #action[:] = act[:]
            act = np.argmax(act)
            #print(act)
            action[act] = 1
            #print(act)
          else:
            act = self.policy_action(curr_state)
            #chosen_action = np.arange(0,self.action_dim)
            #print(chosen_action)
            #chosen_act = np.random.choice(chosen_action,p = act)
            #print(chosen_act)
            action = np.zeros((self.action_dim))
            #action[:] = act[:]
            act = np.argmax(act)
            #print(act)
            action[act] = 1
            #action[chosen_act] = 1
            #act = chosen_act
          #print(act)
          self.epsilon = max(self.epsilon * self.epsilon_decay,0.25)
          #print(self.epsilon)
          (temp,reward,done) = self.env.step(act)
          #print(act,reward)
          next_state[:] = temp[:]
          #print(action)
          #print(curr_state,next_state,reward,done)
          self.memorize(curr_state, action, reward, done, next_state)
          (states, actions, rewards, dones, new_states, _) = self.sample_batch(self.batch_size)
          # Predict Target Q-Values Using Target Network
          q_values = self.critic.target_predict([new_states, self.actor.target_predict(new_states)])
          #print(q_values)
          # Compute Critic Target
          critic_target = self.bellman(rewards, q_values, dones)
          # Train Both Networks on Sampled Batch, Update Target Networks
          self.update_models(states, actions, critic_target)
          # Update Current State
          curr_state[:] = next_state[:]
          cumulative_reward += reward
          steps += 1
          if(steps == 10):
            #print("Fail")
            break
        #print(steps,cumulative_reward)
        #print(steps)
        num_alterations.append(steps)
        cumulative_rewards.append(cumulative_reward)
      return (num_alterations,cumulative_rewards)

    def evaluate(self,max_alterations):
        # Evaluate Agent on Test Data of 5K Malware Instances
        #(self.env.X,self.env.Y) = self.env.univariate_select(self.env.num_features)
        #self.env.X = np.copy(X_new)
        #self.env.Y = np.copy(Y_new)
        self.env.indices = np.where(self.env.Y == 1)
        self.env.data = self.env.X[self.env.indices[0]]
        # Split Malware Instances into Train and Test Data
        #self.env.train_size = int(np.rint(self.env.data.shape[0] * 0.8))
        #self.env.train_data = self.env.data[:self.env.train_size,:]
        #self.env.test_data = self.env.data[self.env.train_size:,:]
        self.env.test_data = self.env.data
        num_alterations = []
        results = []
        poison_data = []
        action_list = []
        max_alterations = min(max_alterations,self.observation_dim)
        #print(max_alterations)
        curr_state = np.zeros((self.observation_dim,))
        next_state = np.zeros((self.observation_dim,))
        input = self.env.test_data
        #print(input.shape)
        #for i in range(0,100):
        for i in range(0,input.shape[0]):
          #print("Malware Instance: ",i,max_alterations)
          steps,done = 0,0
          action_store = []
          curr_state[:] = input[i,:]
          self.env.state = input[i,:]
          #print(self.env.state)
          act = self.policy_action(curr_state)
          indices = act.argsort()
          #print(act)
          #print(indices)
          #index[::-1][:K]
          actions = indices[::-1][:max_alterations]
          #print(actions) 
          #print(curr_state)
          '''while not done:
            #print(curr_state)
            act = self.policy_action(curr_state)
            chosen_action = np.arange(0,self.action_dim)
            chosen_act = np.random.choice(chosen_action,p = act)
            print(act,chosen_act)
            #act = np.argmax(act)
            #(temp,reward,done) = self.env.step(act)
            (temp,reward,done) = self.env.step(chosen_act)
            print(reward)
            #print(temp)
            next_state[:] = temp[:]
            curr_state[:] = next_state[:]
            steps += 1
            if(steps == max_alterations):
              #print("Fail")
              break
          #print(steps)
          #print(curr_state)
          num_alterations.append(steps)
        return num_alterations'''
          for j in range(0,max_alterations):
            #print(self.env.state)
            #print(input[i,:])
            act = actions[j]
            action_store.append(act)
            #print(act)
            (temp,reward,done) = self.env.step(act)
            #print(act,reward,done)
            steps += 1
            if done:
              poison_data.append(self.env.state)
              break
          #malware_data.append(self.env.state)
          num_alterations.append(steps)
          results.append(done)
          action_list.append(action_store)
        poison_data = np.array(poison_data)
        return (num_alterations,results,poison_data,action_list)


    def evaluate_confusion_stats(self,max_alterations):
        # Evaluate Agent on Test Data of 5K Malware Instances
        #(self.env.X,self.env.Y) = self.env.univariate_select(self.env.num_features)
        #self.env.X = np.copy(X_new)
        #self.env.Y = np.copy(Y_new)
        self.env.indices = np.where(self.env.Y == 1)
        self.env.data = self.env.X[self.env.indices[0]]
        # Split Malware Instances into Train and Test Data
        #self.env.train_size = int(np.rint(self.env.data.shape[0] * 0.8))
        #self.env.train_data = self.env.data[:self.env.train_size,:]
        #self.env.test_data = self.env.data[self.env.train_size:,:]
        self.env.test_data = self.env.data
        num_alterations = []
        results = []
        malware_data = []
        action_list = []
        max_alterations = min(max_alterations,self.observation_dim)
        #print(max_alterations)
        curr_state = np.zeros((self.observation_dim,))
        next_state = np.zeros((self.observation_dim,))
        input = self.env.test_data
        #print(input.shape)
        #for i in range(0,100):
        for i in range(0,input.shape[0]):
          #print("Malware Instance: ",i,max_alterations)
          steps,done = 0,0
          action_store = []
          curr_state[:] = input[i,:]
          self.env.state = input[i,:]
          #print(self.env.state)
          act = self.policy_action(curr_state)
          indices = act.argsort()
          #print(act)
          #print(indices)
          #index[::-1][:K]
          actions = indices[::-1][:max_alterations]
          #print(actions) 
          #print(curr_state)
          '''while not done:
            #print(curr_state)
            act = self.policy_action(curr_state)
            chosen_action = np.arange(0,self.action_dim)
            chosen_act = np.random.choice(chosen_action,p = act)
            print(act,chosen_act)
            #act = np.argmax(act)
            #(temp,reward,done) = self.env.step(act)
            (temp,reward,done) = self.env.step(chosen_act)
            print(reward)
            #print(temp)
            next_state[:] = temp[:]
            curr_state[:] = next_state[:]
            steps += 1
            if(steps == max_alterations):
              #print("Fail")
              break
          #print(steps)
          #print(curr_state)
          num_alterations.append(steps)
        return num_alterations'''
          for j in range(0,max_alterations):
            #print(self.env.state)
            #print(input[i,:])
            act = actions[j]
            action_store.append(act)
            #print(act)
            (temp,reward,done) = self.env.step(act)
            #print(act,reward,done)
            steps += 1
            if done:
              #poison_data.append(self.env.state)
              break
          malware_data.append(self.env.state)
          num_alterations.append(steps)
          results.append(done)
          action_list.append(action_store)
        malware_data = np.array(malware_data)
        return (num_alterations,results,malware_data,action_list)
          

    def save_weights(self, path, classifier):
        # Save Weights
        path += '_LR_{}'.format(self.lr)
        path += '_NF_{}'.format(self.observation_dim)
        path += '_'
        path += classifier 
        self.actor.save(path)
        self.critic.save(path)

    def load_weights(self, path_actor, path_critic):
        # Load Weights
        self.critic.load_weights(path_critic)
        self.actor.load_weights(path_actor)

**Classifier Baseline Models**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
baseline_path = path + 'Classifier Baseline Permission/'
num_features_set = [195]
models = ['RF','DT','BNB','SVM','LR','GB','XGB','Stack','DNN1','DNN3','DNN5']
#models = ['DT']
classifiers = {"RF":"Random Forest", "DT":"Decision Tree", "BNB":"Bernoulli Naive Bayes","SVM":"Support Vector Machine","LR":"Logistic Regression","GB":"Gradient Boost","XGB":"Extreme Gradient Boost","Stack":"Stacked - SVM + DT","DNN1":"Neural Network - 1 Hidden Layer","DNN3":"Neural Network - 3 Hidden Layer","DNN5":"Neural Network - 5 Hidden Layer"}
# Workbook is created 
wb = Workbook()   
# add_sheet is used to create sheet. 
sheet1 = wb.add_sheet('Sheet 1')
sheet1.write(0,0,'Classifier')
sheet1.write(0,1,'TP')
sheet1.write(0,2,'TN')
sheet1.write(0,3,'FP')
sheet1.write(0,4,'FN')
sheet1.write(0,5,'Accuracy')
sheet1.write(0,6,'AUC Score')
row = 0
column = 0
for classifier in models:
  method = 'classify' + classifier
  column = 0
  row = row + 1
  sheet1.write(row,column,classifiers.get(classifier))
  model,acc,_,_,_,Y_pred = eval(method)(X_train,X_test,Y_train,Y_test)
  TN,FP,FN,TP = confusion_matrix(Y_test,Y_pred).ravel()
  AUC = roc_auc_score(Y_test, Y_pred)
  column = column + 1
  sheet1.write(row,column,str(TP))
  column = column + 1
  sheet1.write(row,column,str(TN))
  column = column + 1
  sheet1.write(row,column,str(FP))
  column = column + 1
  sheet1.write(row,column,str(FN))
  column = column + 1
  sheet1.write(row,column,str((TP+TN)/(TP+TN+FP+FN)))
  column = column + 1
  sheet1.write(row,column,str(AUC))
  print(classifier, acc, AUC)
  filename = classifier + '_Classifier_NF_' + str(195) + '.pkl' 
  pickle.dump(model, open(filename, 'wb'))
wb.save('Classifier Baseline Stats 70-30 Split Permission.xls')

**Generate Agent Models**

In [ ]:
baseline_path = path + 'Classifier Baseline Permission/'
agent_path = path + 'Agent Permission/'
num_features_set = [195]
num_episodes = 1000
#models = ['RF','DT','BNB','SVM','LR','GB','XGB','Stack','DNN1','DNN3','DNN5']
models = ['RF']
classifiers = {"RF":"Random Forest", "DT":"Decision Tree", "BNB":"Bernoulli Naive Bayes","SVM":"Support Vector Machine","LR":"Logistic Regression","GB":"Gradient Boost","XGB":"Extreme Gradient Boost","Stack":"Stacked","DNN1":"Neural Network - 1 Hidden Layer","DNN3":"Neural Network - 3 Hidden Layer","DNN5":"Neural Network - 5 Hidden Layer"}
for classifier in models:
  method = 'classify' + classifier
  filename = baseline_path + classifier + '_Classifier_NF_' + str(195) + '.pkl'
  model = pickle.load(open(filename, 'rb'))
  for num_feature in num_features_set:
    env = Env(num_feature,model)
    agent = Agent(env)
    #agent.load_weights('/content/_LR_5e-05_NF_195_XGB_actor.h5','/content/_LR_5e-05_NF_195_XGB_critic.h5')
    (num_steps,cumulative_reward) = agent.train(num_episodes)
    agent.save_weights('/content/',classifier)
    agent.env.X = np.copy(X)
    agent.env.Y = np.copy(Y)
    (num_alt,res,poison_data,action_list) = agent.evaluate(10) 
    print(num_feature,10)
    print(sum(res)/len(res))
    print(len(poison_data))
    print(len(action_list))

**Attack Stats**

In [ ]:
baseline_path = path + 'Classifier Baseline Permission/'
agent_path = path + 'Agent Permission/'
num_features = 195
num_episodes = 500
#max_alterations_set = [1,2,3,4,5,6,7,8,9,10]
max_alterations_set = [10]
models = ['RF','DT','BNB','SVM','LR','GB','XGB','Stack','DNN1','DNN3','DNN5']
#models = ['DT','BNB']
classifiers = {"RF":"Random Forest", "DT":"Decision Tree", "BNB":"Bernoulli Naive Bayes","SVM":"Support Vector Machine","LR":"Logistic Regression","GB":"Gradient Boost","XGB":"Extreme Gradient Boost","Stack":"Stacked","DNN1":"Neural Network - 1 Hidden Layer","DNN3":"Neural Network - 3 Hidden Layer","DNN5":"Neural Network - 5 Hidden Layer"}
# Workbook is created 
wb = Workbook()   
for alteration in max_alterations_set:
  alteration_name = 'Original Classifier Fooling Rate - ' + str(alteration) + ' Alterations'
  sheet_name = str(alteration) + ' Bit Attack'
  #print(sheet_name) 
  sheet = wb.add_sheet(sheet_name)
  sheet.write(0,0,'Classifier')
  sheet.write(0,1,'Accuracy')
  sheet.write(0,2,'TP')
  sheet.write(0,3,'TN')
  sheet.write(0,4,'FP')
  sheet.write(0,5,'FN')
  sheet.write(0,6,alteration_name)
  row = 0
  column = 0
  for classifier in models:
    row = row + 1
    column = 0
    sheet.write(row,column,classifiers.get(classifier))
    column = column + 1
    # Load classifier trained on original data
    filename = baseline_path + classifier + '_Classifier_NF_' + str(195) + '.pkl'
    loaded_classifier = pickle.load(open(filename, 'rb'))
    # print(classifier,loaded_classifier.score(X,Y))
    # Load Weights of previously trained Actor Critic Model
    filename = agent_path + '_LR_5e-05_NF_195_' + classifier
    actor_weights = filename + '_actor.h5'
    critic_weights = filename + '_critic.h5'
    env = Env(num_features,loaded_classifier)
    agent = Agent(env)
    agent.load_weights(actor_weights,critic_weights)
    # Perform Adversarial Attack and Generate Poison Data
    (num_alt,res,malware_data,action_list) = agent.evaluate_confusion_stats(alteration)
    if(alteration == 10):
      f = classifier + '_action_list.pkl'
      pickle.dump(action_list, open(f, 'wb'))  
    print(classifier,sum(res)/len(res),alteration)
    benign = X[np.where(Y==0)]
    #benign = benign.reshape(5721,195)
    #print(benign[1])
    #print(malware_data.shape)
    X_new = np.concatenate((X[0:len(benign)],malware_data), axis=0)
    Y_new = np.concatenate((np.zeros((len(benign),1)),np.ones((len(malware_data),1))), axis=0)
    #print(X_new.shape,Y_new.shape)
    acc = loaded_classifier.score(X_new,Y_new)
    sheet.write(row,column,str(acc))
    Y_pred = loaded_classifier.predict(X_new)
    TN,FP,FN,TP = confusion_matrix(Y_new,Y_pred).ravel()
    column = column + 1
    sheet.write(row,column,str(TP))
    column = column + 1
    sheet.write(row,column,str(TN))
    column = column + 1
    sheet.write(row,column,str(FP))
    column = column + 1
    sheet.write(row,column,str(FN))
    column = column + 1
    sheet.write(row,column,str(FN/(FN+TP)))
    #print(classifier)
    print("Accuracy : {} \nTrue Positives : {} \nTrue Negatives : {} \nFalse Positives : {} \nFalse Negatives : {}".format(acc,TP,TN,FP,FN))
wb.save('Attack Stats Permission.xls')

**Classifier Retrained Models**

In [ ]:
baseline_path = path + 'Classifier Baseline Permission/'
agent_path = path + 'Agent Permission/'
models = ['RF','DT','BNB','SVM','LR','GB','XGB','Stack','DNN1','DNN3','DNN5']
#models = ['DT']
classifiers = {"RF":"Random Forest", "DT":"Decision Tree", "BNB":"Bernoulli Naive Bayes","SVM":"Support Vector Machine","LR":"Logistic Regression","GB":"Gradient Boost","XGB":"Extreme Gradient Boost","Stack":"Stacked","DNN1":"Neural Network - 1 Hidden Layer","DNN3":"Neural Network - 3 Hidden Layer","DNN5":"Neural Network - 5 Hidden Layer"}
num_features = 195
max_alteration = 10
# Workbook is created 
wb = Workbook()   
# add_sheet is used to create sheet. 
sheet1 = wb.add_sheet('Sheet 1')
sheet1.write(0,0,'Classifier')
sheet1.write(0,1,'TP')
sheet1.write(0,2,'TN')
sheet1.write(0,3,'FP')
sheet1.write(0,4,'FN')
sheet1.write(0,5,'Accuracy')
sheet1.write(0,6,'AUC Score')
row = 0
column = 0
for classifier in models:
  row = row + 1
  column = 0
  sheet1.write(row,column,classifiers.get(classifier))
  # Load classifier trained on original data
  filename = baseline_path + classifier + '_Classifier_NF_' + str(195) + '.pkl'
  loaded_classifier = pickle.load(open(filename, 'rb'))
  # print(classifier,loaded_classifier.score(X,Y))
  # Load Weights of previously trained Actor Critic Model
  filename = agent_path + '_LR_5e-05_NF_195_' + classifier
  actor_weights = filename + '_actor.h5'
  critic_weights = filename + '_critic.h5'
  env = Env(num_features,loaded_classifier)
  agent = Agent(env)
  agent.load_weights(actor_weights,critic_weights)
  # Perform Adversarial Attack and Generate Poison Data
  (num_alt,res,poison_data,action_list) = agent.evaluate(max_alteration)
  print(sum(res)/len(res))
  #print(len(res))
  poison_data = np.array(poison_data)
  # Perform Oversampling of Benign Instances to Make # Malware and Benign Instances Equal
  benign = X[np.where(Y==0)]
  indices = []
  for i in range(0,len(poison_data)):
    idx = randint(0,len(benign) - 1)
    indices.append(idx)
  # Append Poison Data to Original Data with True Labels and also the Oversampled Benign Instances
  X_new = np.concatenate((X,poison_data), axis=0)
  Y_new = np.concatenate((Y,np.ones((len(poison_data),1))), axis=0)
  X_new = np.concatenate((X_new,X[indices]), axis = 0)
  Y_new = np.concatenate((Y_new,np.zeros((len(X[indices]),1))), axis=0)
  X_train_new, X_test_new, Y_train_new, Y_test_new = train_test_split(X_new, Y_new, test_size=0.30, random_state=42)
  print(X_new.shape,Y_new.shape)
  filename = classifier + '_X_new' 
  pickle.dump(X_new,open(filename,'wb'))
  filename = classifier + '_Y_new'
  pickle.dump(Y_new,open(filename, 'wb'))
  method = 'classify' + classifier
  # Perform Adversarial Retraining
  filename = '/content/' + classifier + '_Retrained_Classifier_NF_' + str(195) + '.pkl'
  retrained_classifier,acc,_,_,_,Y_pred = eval(method)(X_train_new,X_test_new,Y_train_new,Y_test_new)
  TN,FP,FN,TP = confusion_matrix(Y_test_new,Y_pred).ravel()
  AUC = roc_auc_score(Y_test_new, Y_pred)
  column = column + 1
  sheet1.write(row,column,str(TP))
  column = column + 1
  sheet1.write(row,column,str(TN))
  column = column + 1
  sheet1.write(row,column,str(FP))
  column = column + 1
  sheet1.write(row,column,str(FN))
  column = column + 1
  sheet1.write(row,column,str((TP+TN)/(TP+TN+FP+FN)))
  column = column + 1
  sheet1.write(row,column,str(AUC))
  print(retrained_classifier, acc, AUC)
  pickle.dump(retrained_classifier, open(filename, 'wb'))
wb.save('Classifier Retrained Stats 70-30 Split Permission.xls')

**Baseline Class Probabilities**

In [25]:
baseline_path = path + 'Classifier Baseline Permission/'
models = ['RF','DT','BNB','SVM','LR','GB','XGB','Stack','DNN1','DNN3','DNN5']
classifiers = {"RF":"Random Forest", "DT":"Decision Tree", "BNB":"Bernoulli Naive Bayes","SVM":"Support Vector Machine","LR":"Logistic Regression","GB":"Gradient Boost","XGB":"Extreme Gradient Boost","Stack":"Stacked","DNN1":"Neural Network - 1 Hidden Layer","DNN3":"Neural Network - 3 Hidden Layer","DNN5":"Neural Network - 5 Hidden Layer"}
#models = ['DT']
# Workbook is created 
wb = Workbook()   
# add_sheet is used to create sheet. 
sheet1 = wb.add_sheet('Sheet 1')
sheet1.write(0,0,'Classifier')
sheet1.write(0,1,'Average Benign Class Probability')
sheet1.write(0,2,'Average Malware Class Probability')
row = 0
column = 0
num_features = 195
max_alteration = 10
for classifier in models:
  row = row + 1
  column = 0
  sheet1.write(row,column,classifiers.get(classifier))
  column = column + 1
  # Load classifier trained on original data
  filename = baseline_path + classifier + '_Classifier_NF_' + str(195) + '.pkl'
  loaded_classifier = pickle.load(open(filename, 'rb'))
  P = loaded_classifier.predict_proba(X)
  sum_benign = 0
  sum_malware = 0
  for j in range(0,25407):
    sum_benign += P[j][0]
  sheet1.write(row,column,sum_benign/25407)
  column = column + 1
  for j in range(25407,49889):
    sum_malware += P[j][1]
  sheet1.write(row,column,sum_malware/24482)
  print(sum_malware/24482)
  print(sum_benign/25407)
wb.save('Baseline Class Probabilities Permission')

0.9493441248459029
0.9448527570269973
0.9575131446652232
0.9569407503212062
0.8359389790012866
0.9117020916228472
0.9298877639110367
0.9309298362267057
0.893374908912967
0.8968223439820956
0.8777374711413685
0.881531115317107
0.8769535868530673
0.8808891488584544
0.9398492501776095
0.9431398470794855
0.9566939600979883
0.949404029464113
0.9556744243870245
0.9615930605808081
0.9574419332168055
0.9588477658498051


**Retrained Class Probabilities**

In [27]:
baseline_path = path + 'Classifier Retrained Permission/'
data_path = path + 'New Dataset Permission/'
models = ['RF','DT','BNB','SVM','LR','GB','XGB','Stack','DNN1','DNN3','DNN5']
classifiers = {"RF":"Random Forest", "DT":"Decision Tree", "BNB":"Bernoulli Naive Bayes","SVM":"Support Vector Machine","LR":"Logistic Regression","GB":"Gradient Boost","XGB":"Extreme Gradient Boost","Stack":"Stacked","DNN1":"Neural Network - 1 Hidden Layer","DNN3":"Neural Network - 3 Hidden Layer","DNN5":"Neural Network - 5 Hidden Layer"}
#models = ['DT','BNB']
# Workbook is created 
wb = Workbook()   
# add_sheet is used to create sheet. 
sheet1 = wb.add_sheet('Sheet 1')
sheet1.write(0,0,'Classifier')
sheet1.write(0,1,'Average Benign Class Probability')
sheet1.write(0,2,'Average Malware Class Probability')
row = 0
column = 0
total_sum_benign = 0
total_sum_malware = 0
total_benign = 0
total_malware = 0
num_features = 195
max_alteration = 10
for classifier in models:
  row = row + 1
  column = 0
  sheet1.write(row,column,classifiers.get(classifier))
  column = column + 1
  # Load classifier trained on original data
  filename = baseline_path + classifier + '_Retrained_Classifier_NF_' + str(195) + '.pkl'
  loaded_classifier = pickle.load(open(filename, 'rb'))
  filename = data_path + classifier + '_X_new'
  X_new = pickle.load(open(filename, 'rb'))
  filename = data_path + classifier + '_Y_new'
  Y_new = pickle.load(open(filename, 'rb'))
  malware_idx = np.where(Y_new==1)[0]
  benign_idx = np.where(Y_new==0)[0]
  total_malware = total_malware + len(malware_idx)
  total_benign = total_benign + len(benign_idx)
  P = loaded_classifier.predict_proba(X_new)
  sum_benign = 0
  sum_malware = 0
  for j in benign_idx:
    sum_benign += P[j][0]
    total_sum_benign += P[j][0]
  sheet1.write(row,column,sum_benign/len(benign_idx))
  column = column + 1
  for j in malware_idx:
    sum_malware += P[j][1]
    total_sum_malware += P[j][1]
  sheet1.write(row,column,sum_malware/len(malware_idx))
  print(sum_malware/len(malware_idx))
  print(sum_benign/len(benign_idx))
row = row + 1
column = 0
sheet1.write(row,column,'All classifiers')
column = column + 1
sheet1.write(row,column,total_sum_benign/total_benign)
column = column + 1
sheet1.write(row,column,total_sum_malware/total_malware)
wb.save('Retrained Class Probabilities Permission')

0.9595591664508772
0.9605228314571479
0.9708543816469858
0.9729725029353484
0.8467766520643321
0.8971120793922908
0.9487838434418693
0.9506513791479998
0.9112182047754394
0.9142453597901667
0.8852668704190866
0.8879625930601857
0.87705529709693
0.8808906836011762
0.9601745961047906
0.9625611935227055
0.9678884606856191
0.9669297352551994
0.9698112163822773
0.9757101522725746
0.9720367112586951
0.9747327291780339
